In [4]:
import sympy as sp
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

# === Step 1: Generate Functions and Their Taylor Expansions ===
def generate_taylor_dataset(n_samples=100):
    x = sp.Symbol('x')
    functions = [sp.sin(x), sp.cos(x), sp.exp(x), sp.log(1 + x), x**2 + 3*x + 5]
    dataset = []

    for _ in range(n_samples):
        func = random.choice(functions)
        taylor_series = sp.series(func, x, 0, 5).removeO()
        dataset.append((str(func), str(taylor_series)))

    return dataset

dataset = generate_taylor_dataset()

# === Step 2: Tokenize the Dataset ===
def tokenize_expression(expression):
    tokens = list(expression.replace(" ", ""))
    return tokens

all_tokens = set()
for func, taylor in dataset:
    all_tokens.update(tokenize_expression(func))
    all_tokens.update(tokenize_expression(taylor))

token_to_idx = {token: i for i, token in enumerate(sorted(all_tokens))}
idx_to_token = {i: token for token, i in token_to_idx.items()}

def encode_expression(expression, max_len=10):  # Ensure max_len matches LSTM output
    encoded = [token_to_idx[token] for token in tokenize_expression(expression)]
    return encoded[:max_len] + [0] * (max_len - len(encoded))

data_encoded = [(encode_expression(func), encode_expression(taylor)) for func, taylor in dataset]

# === Step 3: Prepare DataLoader for LSTM ===
class TaylorDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]), torch.tensor(self.data[idx][1])

dataset = TaylorDataset(data_encoded)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# === Step 4: LSTM Model ===
class LSTMPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, hidden_dim=64, output_dim=10):  # output_dim = 10
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)  # Ensure output matches target size

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        return self.fc(hidden[-1])  # Output: (batch_size, output_dim)

vocab_size = len(token_to_idx)
lstm_model = LSTMPredictor(vocab_size)
optimizer = optim.Adam(lstm_model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# === Step 5: Train LSTM ===
def train_lstm(model, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)

            print(f"Output shape: {outputs.shape}, Target shape: {targets.shape}")  # Debugging

            loss = criterion(outputs, targets.float())  # Ensure targets are float
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

train_lstm(lstm_model, train_loader)

# === Step 6: Transformer Model ===
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, num_heads=2, hidden_dim=64, output_dim=10):  # output_dim = 10
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.fc = nn.Linear(embed_dim, output_dim)  # Ensure output matches target size

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)  # Change shape for transformer
        x = self.transformer(x)
        x = x.mean(dim=0)  # Take mean over sequence length
        return self.fc(x)  # Output: (batch_size, output_dim)

transformer_model = TransformerModel(vocab_size)
optimizer_trans = optim.Adam(transformer_model.parameters(), lr=0.01)

# === Step 7: Train Transformer ===
def train_transformer(model, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in train_loader:
            optimizer_trans.zero_grad()
            outputs = model(inputs)

            print(f"Output shape: {outputs.shape}, Target shape: {targets.shape}")  # Debugging

            loss = criterion(outputs, targets.float())  # Ensure targets are float
            loss.backward()
            optimizer_trans.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

train_transformer(transformer_model, train_loader)


Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([4, 10]), Target shape: torch.Size([4, 10])
Epoch 1, Loss: 93.42350115094867
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([4, 10]), Target shap

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([4, 10]), Target shape: torch.Size([4, 10])
Epoch 1, Loss: 91.61481693812779
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: torch.Size([4, 10]), Target shape: torch.Size([4, 10])
Epoch 2, Loss: 64.4981302533831
Output shape: torch.Size([16, 10]), Target shape: torch.Size([16, 10])
Output shape: to

In [7]:
import sympy as sp
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

# === Step 1: Generate Functions and Their Taylor Expansions ===
def generate_taylor_dataset(n_samples=100):
    x = sp.Symbol('x')
    functions = [sp.sin(x), sp.cos(x), sp.exp(x), sp.log(1 + x), x**2 + 3*x + 5]
    dataset = []

    for _ in range(n_samples):
        func = random.choice(functions)
        taylor_series = sp.series(func, x, 0, 5).removeO()
        dataset.append((str(func), str(taylor_series)))

    return dataset

dataset = generate_taylor_dataset()

# === Step 2: Tokenize the Dataset ===
def tokenize_expression(expression):
    return list(expression.replace(" ", ""))

# Build vocabulary from dataset
all_tokens = set()
for func, taylor in dataset:
    all_tokens.update(tokenize_expression(func))
    all_tokens.update(tokenize_expression(taylor))

token_to_idx = {token: i for i, token in enumerate(sorted(all_tokens))}
idx_to_token = {i: token for token, i in token_to_idx.items()}

# Convert expressions to numerical sequences
def encode_expression(expression, max_len=10):
    encoded = [token_to_idx[token] for token in tokenize_expression(expression)]
    return encoded[:max_len] + [0] * (max_len - len(encoded))  # Pad/truncate to fixed length

data_encoded = [(encode_expression(func), encode_expression(taylor)) for func, taylor in dataset]

# === Step 3: Prepare DataLoader ===
class TaylorDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_seq, target_seq = self.data[idx]
        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_seq[0], dtype=torch.long)
        # Target is the first token of the Taylor expansion (classification task)

dataset = TaylorDataset(data_encoded)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# === Step 4: LSTM Model ===
class LSTMPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, hidden_dim=64, output_dim=None):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)  # Predicts a token from vocabulary

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        return self.fc(hidden[-1])

vocab_size = len(token_to_idx)
lstm_model = LSTMPredictor(vocab_size)
optimizer = optim.Adam(lstm_model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# === Step 5: Training Function with Accuracy ===
def train_lstm(model, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)  # Shape: [batch_size, vocab_size]

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Accuracy Calculation
            predicted = torch.argmax(outputs, dim=1)  # Get predicted token indices
            correct_predictions += (predicted == targets).sum().item()
            total_predictions += targets.size(0)  # Use batch_size

        accuracy = (correct_predictions / total_predictions) * 100
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

train_lstm(lstm_model, train_loader)

# === Step 6: Transformer Model ===
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, num_heads=2, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.fc = nn.Linear(embed_dim, vocab_size)  # Predicts a token from vocabulary

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)  # Change shape for transformer
        x = self.transformer(x)
        x = x.mean(dim=0)  # Take mean over sequence length
        return self.fc(x)

transformer_model = TransformerModel(vocab_size)
optimizer_trans = optim.Adam(transformer_model.parameters(), lr=0.01)

# === Step 7: Training Transformer with Accuracy ===
def train_transformer(model, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0

        for inputs, targets in train_loader:
            optimizer_trans.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer_trans.step()
            total_loss += loss.item()

            # Accuracy Calculation
            predicted = torch.argmax(outputs, dim=1)
            correct_predictions += (predicted == targets).sum().item()
            total_predictions += targets.size(0)

        accuracy = (correct_predictions / total_predictions) * 100
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

train_transformer(transformer_model, train_loader)


Epoch 1, Loss: 1.8281, Accuracy: 57.00%
Epoch 2, Loss: 0.5733, Accuracy: 71.00%
Epoch 3, Loss: 0.3998, Accuracy: 90.00%
Epoch 4, Loss: 0.1400, Accuracy: 94.00%
Epoch 5, Loss: 0.0164, Accuracy: 100.00%
Epoch 6, Loss: 0.0040, Accuracy: 100.00%
Epoch 7, Loss: 0.0018, Accuracy: 100.00%
Epoch 8, Loss: 0.0011, Accuracy: 100.00%
Epoch 9, Loss: 0.0008, Accuracy: 100.00%
Epoch 10, Loss: 0.0006, Accuracy: 100.00%
Epoch 1, Loss: 1.4472, Accuracy: 61.00%
Epoch 2, Loss: 0.6418, Accuracy: 71.00%
Epoch 3, Loss: 0.6111, Accuracy: 71.00%
Epoch 4, Loss: 0.6020, Accuracy: 71.00%
Epoch 5, Loss: 0.3273, Accuracy: 88.00%
Epoch 6, Loss: 0.0522, Accuracy: 100.00%
Epoch 7, Loss: 0.0158, Accuracy: 100.00%
Epoch 8, Loss: 0.0057, Accuracy: 100.00%
Epoch 9, Loss: 0.0040, Accuracy: 100.00%
Epoch 10, Loss: 0.0026, Accuracy: 100.00%
